__author__ = "Shihao Lin"
__time__   = "2021/11/22"
__version__= "1.0"

In [1]:
!pip install pygame

     |████████████████████████████████| 18.3 MB 17.6 MB/s eta 0:00:01


In [2]:
import pygame
from pygame.surfarray import array3d
import numpy as np
# import os
# chinese_dir = 'Chinese'
# if not os.path.exists(chinese_dir):
#     os.mkdir(chinese_dir)
pygame.init()
font = pygame.font.Font('simfang.ttf',24) #仿宋
font.size('双')

pygame 2.1.0 (SDL 2.0.16, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


(24, 24)

#### Example of a Char to Glyph

In [3]:
rtext = font.render('双',True,(0, 0, 0), (255, 255, 255))
# pygame.image.save(rtext, os.path.join(chinese_dir,'双'+".png"))
rtext = array3d(rtext) # shape: 24,24,3

tmp = np.zeros((576,1),dtype='float32')
for i, rgb in enumerate(rtext.reshape(576,3)):
    # Convert rgb [0,0,0] to 1.      
    tmp[i] = 1-rgb[0]/255.
tmp.shape

(576, 1)

In [9]:
import json
import numpy as np
import pygame
from pygame.surfarray import array3d
def generate_Glyph_initial_weight(chardict_filepath,font_filepath,font_size=24):
    """
    filepath: path to char_map
    return a Glyph Embedding layer weights
        line 0 is reserved for special_tokens like [CLS] [PAD] [SEP]
        line 1 is reserved for [UNK]
    """

    with open(chardict_filepath,'r',encoding='utf-8') as f:
        
        pygame.init()
        font = pygame.font.Font(font_filepath,font_size) #仿宋
        
        char2idx = json.load(f)
        matrix = np.zeros((len(char2idx)+2,font_size**2),dtype='float32')
        matrix[1] = np.random.uniform(-0.1,0.1,font_size**2) # unknown words
        
        for i, char in enumerate(char2idx.keys(),2):
            # shape: font_size,font_size,3
            try:

                char_array = array3d(font.render(char,True,(0,0,0),(255,255,255)))
                # Convert rgb [0,0,0] to 1.
                char_array = 1-char_array[:font_size,:font_size,0].reshape(-1)/255
                matrix[i][:char_array.shape[0]] = char_array
            except:
                print(char)
    return matrix

In [12]:
matrix = generate_Glyph_initial_weight('data/char_map.json','utils/simfang.ttf')

In [58]:
# font = pygame.font.Font('simfang.ttf',25) #仿宋
# a = array3d(font.render('[UNK]',True,(0,0,0),(255,255,255)))

In [14]:
font_weights = np.load('data/glyph_weight.npy').astype(np.float32)

In [13]:
matrix.shape

(9110, 576)

In [15]:
font_weights.shape

(9110, 576)

In [6]:
np.save('glyph_weight',matrix)